# EfficientNetV2M Training for GI Tract Classification

This notebook trains an EfficientNetV2M model on the Kvasir-V2 dataset for gastrointestinal tract image classification.

## Setup Instructions:
1. **Enable GPU**: Runtime → Change runtime type → GPU (T4 or better)
2. **Run all cells sequentially**
3. **Dataset**: Automatically downloads Kvasir-V2 dataset

## 1. Check GPU Availability

In [ ]:
# Check GPU
!nvidia-smi

import tensorflow as tf

print("\nTensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("Num GPUs:", len(tf.config.list_physical_devices('GPU')))

## 2. Install Required Packages

In [ ]:
!pip install -q tensorflow==2.15.0
!pip install -q scikit-learn matplotlib seaborn pandas numpy

print("✅ All packages installed successfully!")

## 3. Download Kvasir-V2 Dataset

In [ ]:
import os
import zipfile
import urllib.request

# Download Kvasir-V2 dataset
dataset_url = "https://datasets.simula.no/downloads/kvasir/kvasir-dataset-v2.zip"
dataset_zip = "kvasir-dataset-v2.zip"

if not os.path.exists("kvasir-dataset-v2"):
    print("Downloading Kvasir dataset...")
    urllib.request.urlretrieve(dataset_url, dataset_zip)
    
    print("Extracting dataset...")
    with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
        zip_ref.extractall(".")
    
    os.remove(dataset_zip)
    print("✅ Dataset ready!")
else:
    print("✅ Dataset already exists!")

# Verify dataset structure
data_dir = "kvasir-dataset-v2/kvasir-dataset-v2/"
classes = sorted(os.listdir(data_dir))
print(f"\nFound {len(classes)} classes: {classes}")

# Count images per class
for cls in classes:
    cls_path = os.path.join(data_dir, cls)
    if os.path.isdir(cls_path):
        num_images = len([f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        print(f"  {cls}: {num_images} images")

## 4. Configure GPU Memory Growth

In [ ]:
# Configure GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ Configured {len(gpus)} GPU(s) with memory growth")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ No GPU found, using CPU")

## 5. Define Constants and Hyperparameters

In [ ]:
# Constants
IMG_SIZE = (224, 224)  # EfficientNetV2M default input size
BATCH_SIZE = 32
NUM_CLASSES = 8
DATA_DIR = "kvasir-dataset-v2/kvasir-dataset-v2/"
EPOCHS = 20
LEARNING_RATE = 1e-5  # Lower learning rate for fine-tuning

print(f"Configuration:")
print(f"  Image Size: {IMG_SIZE}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Num Classes: {NUM_CLASSES}")
print(f"  Epochs: {EPOCHS}")
print(f"  Learning Rate: {LEARNING_RATE}")

## 6. Load and Prepare Dataset

In [ ]:
# Load dataset with train/val/test split
def load_dataset(data_dir, img_size, batch_size):
    # 70% training, 30% for validation + test
    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.3,
        subset="training",
        seed=123,
        image_size=img_size,
        batch_size=batch_size,
    )

    val_test_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.3,
        subset="validation",
        seed=123,
        image_size=img_size,
        batch_size=batch_size,
    )

    # Split val_test_ds into validation and test sets (50/50 split of the 30%)
    val_batches = tf.data.experimental.cardinality(val_test_ds).numpy()
    val_ds = val_test_ds.take(val_batches // 2)
    test_ds = val_test_ds.skip(val_batches // 2)

    return train_ds, val_ds, test_ds

# Preprocessing function
def preprocess_data(image, label):
    image = tf.keras.applications.efficientnet_v2.preprocess_input(image)
    return image, label

# Load datasets
train_ds, val_ds, test_ds = load_dataset(DATA_DIR, IMG_SIZE, BATCH_SIZE)

# Apply preprocessing and optimization
train_ds = train_ds.map(preprocess_data).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.map(preprocess_data).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess_data).prefetch(buffer_size=tf.data.AUTOTUNE)

print("✅ Datasets loaded and preprocessed!")
print(f"  Training batches: {tf.data.experimental.cardinality(train_ds).numpy()}")
print(f"  Validation batches: {tf.data.experimental.cardinality(val_ds).numpy()}")
print(f"  Test batches: {tf.data.experimental.cardinality(test_ds).numpy()}")

## 7. Build EfficientNetV2M Model

In [ ]:
def build_efficientnet_v2_model(num_classes, img_size):
    """Build EfficientNetV2M model with custom classification head"""
    base_model = tf.keras.applications.EfficientNetV2M(
        input_shape=(img_size[0], img_size[1], 3),
        include_top=False,
        weights="imagenet"
    )
    base_model.trainable = True  # Unfreeze for fine-tuning

    inputs = tf.keras.Input(shape=(img_size[0], img_size[1], 3))
    x = tf.keras.applications.efficientnet_v2.preprocess_input(inputs)
    x = base_model(x, training=False)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)
    
    model = tf.keras.Model(inputs, outputs)
    return model

# Build model
model = build_efficientnet_v2_model(NUM_CLASSES, IMG_SIZE)

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

print("✅ Model built and compiled!")
model.summary()

## 8. Define Callbacks

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau

# Define callbacks
callbacks = [
    ModelCheckpoint(
        filepath='efficientnet_v2m_best_model.h5',
        save_weights_only=False,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    CSVLogger(
        'training_log.csv',
        separator=",",
        append=False
    )
]

print("✅ Callbacks configured!")

## 9. Train Model

In [ ]:
print("\n🚀 Starting training...\n")

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=callbacks,
    verbose=1
)

print("\n✅ Training complete!")

## 10. Evaluate on Test Set

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

print("\nEvaluating model on test set...")

# Get predictions
test_labels = np.concatenate([y for x, y in test_ds], axis=0)
test_predictions_probs = model.predict(test_ds)
test_predictions = np.argmax(test_predictions_probs, axis=1)

# Calculate metrics
accuracy = accuracy_score(test_labels, test_predictions)
precision = precision_score(test_labels, test_predictions, average='weighted')
recall = recall_score(test_labels, test_predictions, average='weighted')
f1 = f1_score(test_labels, test_predictions, average='weighted')

print("\n" + "="*60)
print("TEST SET RESULTS")
print("="*60)
print(f"Test Accuracy:  {accuracy:.4f}")
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall:    {recall:.4f}")
print(f"Test F1 Score:  {f1:.4f}")
print("="*60)

# Classification report
class_names = sorted(os.listdir(DATA_DIR))
print("\nClassification Report:")
print(classification_report(test_labels, test_predictions, target_names=class_names))

## 11. Plot Training History

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot loss
axes[0].plot(history.history['loss'], label='Training Loss', marker='o')
axes[0].plot(history.history['val_loss'], label='Validation Loss', marker='s')
axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot accuracy
axes[1].plot(history.history['accuracy'], label='Training Accuracy', marker='o')
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')
axes[1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Training history plot saved!")

## 12. Plot Confusion Matrix

In [ ]:
import seaborn as sns

# Compute confusion matrix
cm = confusion_matrix(test_labels, test_predictions)

# Plot
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - EfficientNetV2M', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Confusion matrix saved!")

## 13. Save Final Results

In [ ]:
import pandas as pd

# Save metrics summary
metrics_summary = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1]
}

df_metrics = pd.DataFrame(metrics_summary)
df_metrics.to_csv('test_metrics_summary.csv', index=False)

print("\n✅ Results saved!")
print("\nFiles created:")
print("  - efficientnet_v2m_best_model.h5 (Best model weights)")
print("  - training_log.csv (Training history)")
print("  - training_history.png (Training plots)")
print("  - confusion_matrix.png (Confusion matrix)")
print("  - test_metrics_summary.csv (Test metrics)")

print("\n" + "="*60)
print("🎉 TRAINING COMPLETE!")
print("="*60)

## 14. Download Results (Optional)

In [ ]:
# Uncomment to download files to your local machine
# from google.colab import files

# files.download('efficientnet_v2m_best_model.h5')
# files.download('training_log.csv')
# files.download('training_history.png')
# files.download('confusion_matrix.png')
# files.download('test_metrics_summary.csv')